In [28]:

from matplotlib import pyplot as plt

import numpy as np
import pandas as pd
import tensorflow as tf
learn = tf.contrib.learn
#tf.enable_eager_execution()

tf.logging.set_verbosity(tf.logging.ERROR)
tf.set_random_seed(123)


## Import the dataset

In [29]:
mnist_local = learn.datasets.load_dataset('mnist')
data = mnist_local.train.images
labels = np.asarray(mnist_local.train.labels, dtype=np.int32)
test_data = mnist_local.test.images
test_labels = np.asarray(mnist_local.test.labels, dtype=np.int32)




Extracting MNIST-data\train-images-idx3-ubyte.gz
Extracting MNIST-data\train-labels-idx1-ubyte.gz
Extracting MNIST-data\t10k-images-idx3-ubyte.gz
Extracting MNIST-data\t10k-labels-idx1-ubyte.gz


In [30]:
max_examples = 10000
data_used_for_training = data[:max_examples]
labels_used_for_training = labels[:max_examples]

In [31]:
x_train = data_used_for_training
y_train = labels_used_for_training  
x_test  = data[10000:12000]
y_test  = labels[10000:12000]

In [32]:
print ("Traning set feature shape: ", x_train.shape,"\n")
print ("Traning set label shape: ", y_train.shape,"\n")
print ("Testing set feature shape: ", x_test.shape,"\n")
print ("Testing set label shape: ", y_test.shape,"\n")

Traning set feature shape:  (10000, 784) 

Traning set label shape:  (10000,) 

Testing set feature shape:  (2000, 784) 

Testing set label shape:  (2000,) 



In [33]:
y_train[0:10]
x_train[0]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [34]:
# noramlized features from 0-1  
x_train, x_test = x_train / 255.0, x_test / 255.0
# define input_size
dim=28

In [35]:
# Define the training inputs
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_train},
    y=y_train.astype(np.int32),
    num_epochs=None,
    batch_size=50,
    shuffle=True
)

In [36]:
feature_columns = [tf.feature_column.numeric_column("x", shape=[28, 28])]

In [37]:
print ( feature_columns)

[_NumericColumn(key='x', shape=(28, 28), default_value=None, dtype=tf.float32, normalizer_fn=None)]


There are 55k examples in train, and 10k in eval. You may wish to limit the size to experiment faster. 

In [38]:
#Start training the model

sess = tf.InteractiveSession()
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[256],
    optimizer=tf.train.AdamOptimizer(1e-4),
    n_classes=10,
    dropout=0.2,
    model_dir="./mnist_model"
)

In [39]:
import shutil
shutil.rmtree("./mnist_model", ignore_errors = True)
classifier.train(input_fn=train_input_fn, steps=30000)

In [40]:
# Define the test inputs
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_test},
    y=y_test.astype(np.int32),
    num_epochs=1,
    shuffle=False
)

In [41]:
# Evaluate accuracy
accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]
print("\nTest Accuracy: {0:f}%\n".format(accuracy_score*100))


Test Accuracy: 89.850003%



In [44]:
data_used_for_testing = data[max_examples:15000]
labels_used_for_testing= labels[max_examples:15000]

## Display some digits

In [45]:
def display(i):
    img = test_data[i]
    plt.title('Example %d. Label: %d' % (i, test_labels[i]))
    plt.imshow(img.reshape((28,28)), cmap=plt.cm.gray_r)    

In [46]:
display(0)

In [47]:
display(1)

These digits are clearly drawn. Here's one that's not.

In [48]:
display(8)

Now let's take a look at how many features we have.

In [49]:
print ( len(data[0]))



784


## Evaluate accuracy

## Classify a few examples

We can make predictions on individual images using the predict method

builder = tf.saved_model.builder.SavedModelBuilder("./model")

signature = predict_signature_def(inputs={'myInput': x},
                                  outputs={'myOutput': y})
builder.add_meta_graph_and_variables(sess=sess,
                                     tags=[tag_constants.SERVING],
                                     signature_def_map={'predict': signature})
builder.save()

## Visualize learned weights


Let's see if we can reproduce the pictures of the weights in the TensorFlow Basic MNSIT <a href="https://www.tensorflow.org/tutorials/mnist/beginners/index.html#mnist-for-ml-beginners">tutorial</a>. 

builder = tf.saved_model.builder.SavedModelBuilder('model6')
signature = predict_signature_def(inputs={'input': x}, outputs={'output': y})
builder.add_meta_graph_and_variables(sess, [tf.saved_model.tag_constants.SERVING],signature_def_map={'predict': signature})
builder.save()

In [50]:
# Export model
# WARNING(break-tutorial-inline-code): The following code snippet is
# in-lined in tutorials, please update tutorial documents accordingly
# whenever code changes.
import os

export_path_base = "./"
export_path = os.path.join(
  tf.compat.as_bytes(export_path_base),
  tf.compat.as_bytes("3"))

print('Exporting trained model to', export_path)
builder = tf.saved_model.builder.SavedModelBuilder(export_path)

serialized_tf_example = tf.placeholder(tf.string, name='tf_example')
feature_configs = {'x': tf.FixedLenFeature(shape=[784], dtype=tf.float32),}
tf_example = tf.parse_example(serialized_tf_example, feature_configs)


# Build the signature_def_map.
#classification_inputs = tf.saved_model.utils.build_tensor_info(serialized_tf_example)
#classification_outputs_classes = tf.saved_model.utils.build_tensor_info(prediction_classes)
#classification_outputs_scores = tf.saved_model.utils.build_tensor_info(values)

xx = tf.convert_to_tensor(x_train)
yy = tf.convert_to_tensor(y_train)

tensor_info_x = tf.saved_model.utils.build_tensor_info(xx)
tensor_info_y = tf.saved_model.utils.build_tensor_info(yy)

prediction_signature = (
  tf.saved_model.signature_def_utils.build_signature_def(
      inputs={'images': tensor_info_x},
      outputs={'scores': tensor_info_y},
      method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME))

legacy_init_op = tf.group(tf.tables_initializer(), name='legacy_init_op')
builder.add_meta_graph_and_variables(
  sess, [tf.saved_model.tag_constants.SERVING],
  signature_def_map={
      'predict_images':
          prediction_signature
  },
  legacy_init_op=legacy_init_op)

builder.save()

print('Done exporting!')

Exporting trained model to b'./3'
Done exporting!
